In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.decomposition import PCA
import cv2

In [2]:
fullyrestrained = ['20220224','20210712','20210928','20210816','20220203','20210710','20220309pipe','20220210pipe','20220210uc','20220309stick']
semirestrained = ['20210921','20111108','20220405uc','20220405pipe','20210830','20211009']

In [22]:
base_dir = '/Volumes/L_MillerLab/data/Pop_18E3/Jarvis/all_fr/'
jarvis_dict = {'FR': {}, 'SR':{}}
for date_csv in os.listdir(base_dir):
    date = date_csv.split('_')[0]
    print(date)
    if date in semirestrained:
        csv = np.genfromtxt(base_dir + date_csv, delimiter = ',')
        jarvis_dict['SR'][date] = csv[2:,:]
    else:
        csv = np.genfromtxt(base_dir + date_csv, delimiter = ',')
        jarvis_dict['FR'][date] = csv[2:,:]

20211105
20211116uc
20210712
20210928
20211119uc
20210710
20220405uc
20211209
20211207uc
20211203
20220224
20211206uc
20220309stick
20211119stick
20210814
20211116stick
20211206pipe


In [23]:
jarvis_dict['FR']['20220224'].shape

(18462, 92)

In [30]:
jarvis_dict_nonans = {'FR': {}, 'SR':{}}
for key in jarvis_dict.keys():
    print(key)
    for date in jarvis_dict[key]:
        nan_rows = np.isnan(jarvis_dict[key][date]).any(axis=1)
        nan_rows_indices=np.where(nan_rows)[0]
        print(date, jarvis_dict[key][date].shape, nan_rows_indices.shape)
#         if nan_rows_indices.shape[0]<500:
#             print(nan_rows_indices.shape[0],nan_rows_indices.shape[0]<500)
        arr = np.copy(jarvis_dict[key][date])
        for j in range(arr.shape[1]):
            # check if the column contains any NaN values
            if np.isnan(arr[:,j]).any():
                # if the column contains NaN values, interpolate the missing values
                x = np.arange(arr.shape[0])
                y = arr[:,j]
                y_interp = np.interp(x, x[~np.isnan(y)], y[~np.isnan(y)])
                arr[:,j] = np.where(np.isnan(y), y_interp, y)

        jarvis_dict_nonans[key][date] = arr

FR
20211105 (36928, 92) (4,)
20211116uc (9279, 104) (447,)
20210712 (37453, 92) (13,)
20210928 (37197, 104) (399,)
20211119uc (18215, 104) (661,)
20210710 (54821, 104) (5273,)
20211209 (18776, 104) (952,)
20211207uc (18570, 104) (24,)
20211203 (18561, 104) (91,)
20220224 (18462, 92) (0,)
20211206uc (27420, 104) (3026,)
20220309stick (17998, 92) (0,)
20211119stick (18498, 104) (1863,)
20210814 (55372, 92) (1613,)
20211116stick (36000, 104) (1633,)
20211206pipe (9227, 104) (680,)
SR
20220405uc (17998, 92) (444,)


In [31]:
for key in jarvis_dict.keys():
    print(key)
    for date in jarvis_dict[key]:
        nan_rows = np.isnan(jarvis_dict[key][date]).any(axis=1)
        nan_rows_indices=np.where(nan_rows)[0]
        print(date, jarvis_dict[key][date].shape, nan_rows_indices.shape)

FR
20211105 (36928, 92) (4,)
20211116uc (9279, 104) (447,)
20210712 (37453, 92) (13,)
20210928 (37197, 104) (399,)
20211119uc (18215, 104) (661,)
20210710 (54821, 104) (5273,)
20211209 (18776, 104) (952,)
20211207uc (18570, 104) (24,)
20211203 (18561, 104) (91,)
20220224 (18462, 92) (0,)
20211206uc (27420, 104) (3026,)
20220309stick (17998, 92) (0,)
20211119stick (18498, 104) (1863,)
20210814 (55372, 92) (1613,)
20211116stick (36000, 104) (1633,)
20211206pipe (9227, 104) (680,)
SR
20220405uc (17998, 92) (444,)


In [32]:
for key in jarvis_dict_nonans.keys():
    print(key)
    for date in jarvis_dict_nonans[key]:
        nan_rows = np.isnan(jarvis_dict_nonans[key][date]).any(axis=1)
        nan_rows_indices=np.where(nan_rows)[0]
        print(date, jarvis_dict_nonans[key][date].shape, nan_rows_indices.shape)

FR
20211105 (36928, 92) (0,)
20211116uc (9279, 104) (0,)
20210712 (37453, 92) (0,)
20210928 (37197, 104) (0,)
20211119uc (18215, 104) (0,)
20210710 (54821, 104) (0,)
20211209 (18776, 104) (0,)
20211207uc (18570, 104) (0,)
20211203 (18561, 104) (0,)
20220224 (18462, 92) (0,)
20211206uc (27420, 104) (0,)
20220309stick (17998, 92) (0,)
20211119stick (18498, 104) (0,)
20210814 (55372, 92) (0,)
20211116stick (36000, 104) (0,)
20211206pipe (9227, 104) (0,)
SR
20220405uc (17998, 92) (0,)


In [33]:
outlier_indices = {}
for key in jarvis_dict_nonans.keys():
    outlier_indices[key] = {}
    for date in jarvis_dict_nonans[key]:
        data = jarvis_dict_nonans[key][date]
        
        pinky_t = data[:,:3]
        pinky_d = data[:,4:7]
        pinky_m = data[:,8:11]
        pinky_p = data[:,12:15]
        ring_t = data[:,16:19]
        ring_d = data[:,20:23]
        ring_m = data[:,24:27]
        ring_p = data[:,28:31]
        middle_t = data[:,32:35]
        middle_d = data[:,36:39]
        middle_m = data[:,40:43]
        middle_p = data[:,44:47]
        index_t = data[:,48:51]
        index_d = data[:,52:55]
        index_m = data[:,56:59]
        index_p = data[:,60:63]
        thumb_t = data[:,64:67]
        thumb_d = data[:,68:71]
        thumb_m = data[:,72:75]
        thumb_p = data[:,76:79]
        
        pinky_dist_1 = pinky_t-pinky_d
        pinky_dist_2 = pinky_d-pinky_m
        pinky_dist_3 = pinky_m-pinky_p
        ring_dist_1 = ring_t-ring_d
        ring_dist_2 = ring_d-ring_m
        ring_dist_3 = ring_m-ring_p
        middle_dist_1 = middle_t-middle_d
        middle_dist_2 = middle_d-middle_m
        middle_dist_3 = middle_m-middle_p
        index_dist_1 = index_t-index_d
        index_dist_2 = index_d-index_m
        index_dist_3 = index_m-index_p
        thumb_dist_1 = thumb_t-thumb_d
        thumb_dist_2 = thumb_d-thumb_m
        thumb_dist_3 = thumb_m-thumb_p

        finger_dist = np.hstack([pinky_dist_1,pinky_dist_2,pinky_dist_3,ring_dist_1,ring_dist_2,ring_dist_3,middle_dist_1,middle_dist_2,middle_dist_3,index_dist_1,index_dist_2,index_dist_3,thumb_dist_1,thumb_dist_2,thumb_dist_3])
        
        pca = PCA(n_components=1)
        finger_dist_pc1 = pca.fit_transform(finger_dist)
        
        # calculate the mean and standard deviation of the array
        mean = np.median(finger_dist_pc1)
        std = np.std(finger_dist_pc1)

        # calculate the lower and upper bounds for the values within 2 standard deviations of the mean
        lower_bound = mean - 2 * std
        upper_bound = mean + 2 * std

        # find the indices of the values that do not fall within the bounds
        indices = np.where((finger_dist_pc1 < lower_bound) | (finger_dist_pc1 > upper_bound))[0]
        print(date, finger_dist_pc1.shape[0], indices.shape[0], round(indices.shape[0]/finger_dist_pc1.shape[0],3))
        outlier_indices[key][date] = indices

20211105 36928 8 0.0
20211116uc 9279 507 0.055
20210712 37453 672 0.018
20210928 37197 431 0.012
20211119uc 18215 1024 0.056
20210710 54821 2533 0.046
20211209 18776 1669 0.089
20211207uc 18570 1507 0.081
20211203 18561 1683 0.091
20220224 18462 1780 0.096
20211206uc 27420 1661 0.061
20220309stick 17998 0 0.0
20211119stick 18498 1033 0.056
20210814 55372 22 0.0
20211116stick 36000 2289 0.064
20211206pipe 9227 695 0.075
20220405uc 17998 1580 0.088


In [40]:
total_errors = {'FR': {}, 'SR': {}}
total_errors_outliers = {'FR': {}, 'SR': {}}
speeds = {'FR': {}, 'SR': {}}
for key in jarvis_dict_nonans.keys():
    for date in jarvis_dict_nonans[key]:
        data = jarvis_dict_nonans[key][date]

        elect_mean = np.mean(data, axis = 0)
        diff = data-elect_mean

        #get three columns for each type
        pinky_t = data[:,:3]
        pinky_t_velocity = np.diff(pinky_t, axis = 0)*30
        pinky_t_speed = np.sqrt(np.sum(pinky_t_velocity**2, axis =1))
        pinky_d = data[:,4:7]
        pinky_d_velocity = np.diff(pinky_d, axis = 0)*30
        pinky_d_speed = np.sqrt(np.sum(pinky_d_velocity**2, axis =1))
        pinky_m = data[:,8:11]
        pinky_m_velocity = np.diff(pinky_m, axis = 0)*30
        pinky_m_speed = np.sqrt(np.sum(pinky_m_velocity**2, axis =1))
        pinky_p = data[:,12:15]
        pinky_p_velocity = np.diff(pinky_p, axis = 0)*30
        pinky_p_speed = np.sqrt(np.sum(pinky_p_velocity**2, axis =1))
        ring_t = data[:,16:19]
        ring_t_velocity = np.diff(ring_t, axis = 0)*30
        ring_t_speed = np.sqrt(np.sum(ring_t_velocity**2, axis =1))
        ring_d = data[:,20:23]
        ring_d_velocity = np.diff(ring_d, axis = 0)*30
        ring_d_speed = np.sqrt(np.sum(ring_d_velocity**2, axis =1))
        ring_m = data[:,24:27]
        ring_m_velocity = np.diff(ring_m, axis = 0)*30
        ring_m_speed = np.sqrt(np.sum(ring_m_velocity**2, axis =1))
        ring_p = data[:,28:31]
        ring_p_velocity = np.diff(ring_p, axis = 0)*30
        ring_p_speed = np.sqrt(np.sum(ring_p_velocity**2, axis =1))
        middle_t = data[:,32:35]
        middle_t_velocity = np.diff(middle_t, axis = 0)*30
        middle_t_speed = np.sqrt(np.sum(middle_t_velocity**2, axis =1))
        middle_d = data[:,36:39]
        middle_d_velocity = np.diff(middle_d, axis = 0)*30
        middle_d_speed = np.sqrt(np.sum(middle_d_velocity**2, axis =1))
        middle_m = data[:,40:43]
        middle_m_velocity = np.diff(middle_m, axis = 0)*30
        middle_m_speed = np.sqrt(np.sum(middle_m_velocity**2, axis =1))
        middle_p = data[:,44:47]
        middle_p_velocity = np.diff(middle_p, axis = 0)*30
        middle_p_speed = np.sqrt(np.sum(middle_p_velocity**2, axis =1))
        index_t = data[:,48:51]
        index_t_velocity = np.diff(index_t, axis = 0)*30
        index_t_speed = np.sqrt(np.sum(index_t_velocity**2, axis =1))
        index_d = data[:,52:55]
        index_d_velocity = np.diff(index_d, axis = 0)*30
        index_d_speed = np.sqrt(np.sum(index_d_velocity**2, axis =1))
        index_m = data[:,56:59]
        index_m_velocity = np.diff(index_m, axis = 0)*30
        index_m_speed = np.sqrt(np.sum(index_m_velocity**2, axis =1))
        index_p = data[:,60:63]
        index_p_velocity = np.diff(index_p, axis = 0)*30
        index_p_speed = np.sqrt(np.sum(index_p_velocity**2, axis =1))
        thumb_t = data[:,64:67]
        thumb_t_velocity = np.diff(thumb_t, axis = 0)*30
        thumb_t_speed = np.sqrt(np.sum(thumb_t_velocity**2, axis =1))
        thumb_d = data[:,68:71]
        thumb_d_velocity = np.diff(thumb_d, axis = 0)*30
        thumb_d_speed = np.sqrt(np.sum(thumb_d_velocity**2, axis =1))
        thumb_m = data[:,72:75]
        thumb_m_velocity = np.diff(thumb_m, axis = 0)*30
        thumb_m_speed = np.sqrt(np.sum(thumb_m_velocity**2, axis =1))
        thumb_p = data[:,76:79]
        thumb_p_velocity = np.diff(thumb_p, axis = 0)*30
        thumb_p_speed = np.sqrt(np.sum(thumb_p_velocity**2, axis =1))

        pinky_dist_1 = pinky_t-pinky_d
        pinky_dist_1_mean = np.mean(pinky_dist_1, axis = 0)
        pinky_dist_1_error = np.sum((pinky_dist_1_mean - pinky_dist_1)**2, axis=1)
        pinky_dist_2 = pinky_d-pinky_m
        pinky_dist_2_mean = np.mean(pinky_dist_2, axis = 0)
        pinky_dist_2_error = np.sum((pinky_dist_2_mean - pinky_dist_2)**2, axis=1)
        pinky_dist_3 = pinky_m-pinky_p
        pinky_dist_3_mean = np.mean(pinky_dist_3, axis = 0)
        pinky_dist_3_error = np.sum((pinky_dist_3_mean - pinky_dist_3)**2, axis=1)
        ring_dist_1 = ring_t-ring_d
        ring_dist_1_mean = np.mean(ring_dist_1, axis = 0)
        ring_dist_1_error = np.sum((ring_dist_1_mean - ring_dist_1)**2, axis=1)
        ring_dist_2 = ring_d-ring_m
        ring_dist_2_mean = np.mean(ring_dist_2, axis = 0)
        ring_dist_2_error = np.sum((ring_dist_2_mean - ring_dist_2)**2, axis=1)
        ring_dist_3 = ring_m-ring_p
        ring_dist_3_mean = np.mean(ring_dist_3, axis = 0)
        ring_dist_3_error = np.sum((ring_dist_3_mean - ring_dist_3)**2, axis=1)
        middle_dist_1 = middle_t-middle_d
        middle_dist_1_mean = np.mean(middle_dist_1, axis = 0)
        middle_dist_1_error = np.sum((middle_dist_1_mean - middle_dist_1)**2, axis=1)
        middle_dist_2 = middle_d-middle_m
        middle_dist_2_mean = np.mean(middle_dist_2, axis = 0)
        middle_dist_2_error = np.sum((middle_dist_2_mean - middle_dist_2)**2, axis=1)
        middle_dist_3 = middle_m-middle_p
        middle_dist_3_mean = np.mean(middle_dist_3, axis = 0)
        middle_dist_3_error = np.sum((middle_dist_3_mean - middle_dist_3)**2, axis=1)
        index_dist_1 = index_t-index_d
        index_dist_1_mean = np.mean(index_dist_1, axis = 0)
        index_dist_1_error = np.sum((index_dist_1_mean - index_dist_1)**2, axis=1)
        index_dist_2 = index_d-index_m
        index_dist_2_mean = np.mean(index_dist_2, axis = 0)
        index_dist_2_error = np.sum((index_dist_2_mean - index_dist_2)**2, axis=1)
        index_dist_3 = index_m-index_p
        index_dist_3_mean = np.mean(index_dist_3, axis = 0)
        index_dist_3_error = np.sum((index_dist_3_mean - index_dist_3)**2, axis=1)
        thumb_dist_1 = thumb_t-thumb_d
        thumb_dist_1_mean = np.mean(thumb_dist_1, axis = 0)
        thumb_dist_1_error = np.sum((thumb_dist_1_mean - thumb_dist_1)**2, axis=1)
        thumb_dist_2 = thumb_d-thumb_m
        thumb_dist_2_mean = np.mean(thumb_dist_2, axis = 0)
        thumb_dist_2_error = np.sum((thumb_dist_2_mean - thumb_dist_2)**2, axis=1)
        thumb_dist_3 = thumb_m-thumb_p
        thumb_dist_3_mean = np.mean(thumb_dist_3, axis = 0)
        thumb_dist_3_error = np.sum((thumb_dist_3_mean - thumb_dist_3)**2, axis=1)

        total_error = (pinky_dist_1_error+pinky_dist_2_error+pinky_dist_3_error+
                       ring_dist_1_error+ring_dist_2_error+ring_dist_3_error+
                       middle_dist_1_error+middle_dist_2_error+middle_dist_3_error+
                       index_dist_1_error+index_dist_2_error+index_dist_3_error+
                       thumb_dist_1_error+thumb_dist_2_error+thumb_dist_3_error)/15
        average_speed = (pinky_t_speed+pinky_d_speed+pinky_m_speed+pinky_p_speed+
                         ring_t_speed+ring_d_speed+ring_m_speed+ring_p_speed+
                         middle_t_speed+middle_d_speed+middle_m_speed+middle_p_speed+
                         index_t_speed+index_d_speed+index_m_speed+index_p_speed+
                         thumb_t_speed+thumb_d_speed+thumb_m_speed+thumb_p_speed)/20
        print(key, date, round(np.mean(total_error), 3), total_error.shape, average_speed.shape)
        
        sorted_indices = np.argsort(-total_error)
        num_top_indices = int(0.05 * len(total_error))
        top_indices = sorted_indices[:num_top_indices]

        total_errors[key][date] = total_error
        total_errors_outliers[key][date] = sorted(top_indices)
        speeds[key][date] = average_speed

FR 20211105 280.877 (36928,) (36927,)
FR 20211116uc 2527.59 (9279,) (9278,)
FR 20210712 286.694 (37453,) (37452,)
FR 20210928 362.392 (37197,) (37196,)
FR 20211119uc 920.846 (18215,) (18214,)
FR 20210710 462.345 (54821,) (54820,)
FR 20211209 1409.08 (18776,) (18775,)
FR 20211207uc 920.879 (18570,) (18569,)
FR 20211203 1537.354 (18561,) (18560,)
FR 20220224 212.083 (18462,) (18461,)
FR 20211206uc 1399.186 (27420,) (27419,)
FR 20220309stick 258.793 (17998,) (17997,)
FR 20211119stick 761.619 (18498,) (18497,)
FR 20210814 419.058 (55372,) (55371,)
FR 20211116stick 1712.514 (36000,) (35999,)
FR 20211206pipe 1166.602 (9227,) (9226,)
SR 20220405uc 580.88 (17998,) (17997,)


In [35]:
for key in total_errors_outliers:
    for date in total_errors_outliers[key]:
        print(date, len(total_errors_outliers[key][date]),len(outlier_indices[key][date]))

20211105 1846 8
20211116uc 463 507
20210712 1872 672
20210928 1859 431
20211119uc 910 1024
20210710 2741 2533
20211209 938 1669
20211207uc 928 1507
20211203 928 1683
20220224 923 1780
20211206uc 1371 1661
20220309stick 899 0
20211119stick 924 1033
20210814 2768 22
20211116stick 1800 2289
20211206pipe 461 695
20220405uc 899 1580


In [36]:
d = {}
for key in total_errors_outliers:
    d[key] = {}
    for date in total_errors_outliers[key]:
        d[key][date] = {}
        total_errors_outliers[key][date],outlier_indices[key][date]
        total_errors_outliers_set = set(total_errors_outliers[key][date])
        outlier_indices_set = set(outlier_indices[key][date])

        common_values = outlier_indices_set.intersection(total_errors_outliers_set)
        only_pc_method = outlier_indices_set.difference(total_errors_outliers_set)
        only_mse_method = total_errors_outliers_set.difference(outlier_indices_set)
        
        d[key][date]['common_values'] = common_values
        d[key][date]['only_pc_method'] = only_pc_method
        d[key][date]['only_mse_method'] = only_mse_method

# Saving Frames

In [37]:
for key in d:
    for date in d[key]:
        for val in d[key][date]:
            print(key, date, val,len(d[key][date][val]))

FR 20211105 common_values 6
FR 20211105 only_pc_method 2
FR 20211105 only_mse_method 1840
FR 20211116uc common_values 131
FR 20211116uc only_pc_method 376
FR 20211116uc only_mse_method 332
FR 20210712 common_values 247
FR 20210712 only_pc_method 425
FR 20210712 only_mse_method 1625
FR 20210928 common_values 75
FR 20210928 only_pc_method 356
FR 20210928 only_mse_method 1784
FR 20211119uc common_values 312
FR 20211119uc only_pc_method 712
FR 20211119uc only_mse_method 598
FR 20210710 common_values 981
FR 20210710 only_pc_method 1552
FR 20210710 only_mse_method 1760
FR 20211209 common_values 590
FR 20211209 only_pc_method 1079
FR 20211209 only_mse_method 348
FR 20211207uc common_values 604
FR 20211207uc only_pc_method 903
FR 20211207uc only_mse_method 324
FR 20211203 common_values 702
FR 20211203 only_pc_method 981
FR 20211203 only_mse_method 226
FR 20220224 common_values 841
FR 20220224 only_pc_method 939
FR 20220224 only_mse_method 82
FR 20211206uc common_values 788
FR 20211206uc only_p

In [38]:
# vis_dates = ['20211009', '20211108', '20220309stick', '20220405uc']
# vis_dates = ['20220224', '20211203', '20210928', '20211119stick']
vis_dates = ['20211119stick','20211203']

base_vis_dir = '/Volumes/L_MillerLab/limblab/User_folders/Aajan/visualization/'
for key in d:
    for date in d[key]:
        print(date)
        if date in vis_dates:
            print(key, date)
            video_path = os.path.join(base_vis_dir,date,'cam_0.mp4')
            video_capture = cv2.VideoCapture(video_path)
            print(video_path)
            for val in d[key][date].keys():
                full_dir = os.path.join(base_vis_dir,date,val)
#                 print(full_dir)
                if os.path.exists(full_dir) == False:
                    os.makedirs(full_dir)
                
                print(key, date, val,len(d[key][date][val]))
                if len(d[key][date][val])>0:
                    
                    for index in d[key][date][val]:
                        print(index)
                        video_capture.set(cv2.CAP_PROP_POS_FRAMES, index)

                        # Read the frame
                        ret, frame = video_capture.read()

                        # Save the frame as an image
                        if ret:
                            cv2.imwrite(os.path.join(full_dir,'frame_{}.jpg'.format(index)), frame)
                        else:
                            print(f'Error reading frame {index} from the video')

            # Release the video capture object
            video_capture.release()

20211105
20211116uc
20210712
20210928
20211119uc
20210710
20211209
20211207uc
20211203
FR 20211203
/Volumes/L_MillerLab/limblab/User_folders/Aajan/visualization/20211203/cam_0.mp4
FR 20211203 common_values 702
16394
16402
15477
14414
18525
18536
16544
16550
16579
16584
16585
16590
16596
16597
16600
16601
14569
14570
16618
14572
16619
14575
16639
16645
16652
16654
16658
14611
14612
16662
16663
6425
6426
6428
6430
14631
14633
14637
16686
16693
14653
14656
14659
14660
14664
16712
14667
14668
14669
14679
14680
14681
14683
16736
16749
16751
16754
14713
16802
16806
16807
16845
16846
16863
16865
16867
16868
16870
16890
16895
16911
14865
14870
14872
16922
16923
16924
14878
14880
16928
14884
14885
14886
16935
14888
14890
16939
14892
16943
16945
14908
14909
14910
14911
14913
14915
14916
14917
14930
14931
14932
14933
14934
14935
14936
14937
14938
14939
14940
14941
14943
14944
14945
14946
14947
14948
14949
14950
14951
14952
14953
14954
14955
14956
14958
14959
14960
14962
14964
14965
14966
14967
14

17771
17776
17777
17778
9592
5497
17789
17791
17793
13698
17795
17796
17797
13701
17799
17800
13705
13713
13716
13718
13723
17820
13727
17825
13729
13731
13733
13737
13738
13740
17837
13741
13744
17844
17846
13752
13756
13757
13760
13762
13763
13764
13765
17862
13766
13768
13769
13770
13771
13773
17870
13775
17872
13778
17875
17877
17878
13781
17880
13782
17884
17885
13789
17888
13795
17896
17897
17899
17900
17901
17902
13806
17904
13808
17906
17913
17915
17919
13824
17921
3588
17925
13829
9738
17934
17935
17936
17940
17941
17942
17949
17952
17953
17957
13863
17960
17961
17971
17975
17977
17979
17981
17985
17987
17991
17992
17993
17994
13897
17998
15950
18000
18001
18003
18009
18010
18011
18013
13919
18016
18020
18021
18022
13924
15973
18025
15975
18027
18029
18031
13935
15983
18038
13947
18044
13948
18046
13949
18048
15998
18051
18053
18055
18056
18057
18058
18059
18060
16010
13966
18063
16014
16017
18066
18067
18068
18070
18071
18073
18074
18075
18076
18077
18081
16035
18088
18094
14

1306
15647
15648
15669
15670
15671
15673
13630
17733
3400
13645
13646
13647
13649
15697
15698
15702
15703
15704
15705
15706
11611
17756
17757
15707
15708
15709
15710
11617
15711
3427
3428
15712
17763
1395
11638
3448
11642
15742
15743
15746
15748
11660
1422
13710
13711
13729
9634
9635
7589
7590
11696
11697
13746
11699
15792
15797
15800
17853
17856
17857
13772
13778
13784
17881
17899
13809
13810
11763
16749
11766
11767
11768
11769
11770
15873
15874
11779
17923
1546
1547
11790
15896
15906
7721
17964
17965
17966
17967
17968
17969
17971
17975
17980
17981
13888
13889
13890
17989
13894
17990
17992
17993
17994
17995
13896
13897
13898
17999
13899
18001
18002
18003
18004
18005
9805
13902
3666
9812
9813
13907
13908
13911
13916
18009
13920
11888
11889
1652
13941
15995
15999
16000
16002
16007
16018
13971
11924
13972
9878
13973
7836
13981
1714
1715
1716
11967
3777
16065
16067
11972
16068
16069
16070
16071
16072
16073
16075
16084
16087
3803
3804
16094
3811
14056
14057
14058
16109
16121
14075
14078
16